
#### Speech Denoising using LSTM

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import librosa
import numpy as np
import random
import glob
import os
import pickle

In [0]:
def filegen(path,picklepath):
  train = []
  for f in sorted(glob.glob(path)):
    s, sr=librosa.load(f, sr=None)
    S=librosa.stft(s, n_fft=1024, hop_length=512)
    train.append(np.abs(S).T)
  with open(picklepath, 'wb') as fi:
    pickle.dump(train,fi)
  return(train)


def validfilegen(path,picklepath):
  valid = []
  if(path == '/content/drive/My Drive/homework3/timit-homework/v/vx*'):
    for f1 in sorted(glob.glob(path)):
      s, sr=librosa.load(f1, sr=None)
      S=librosa.stft(s, n_fft=1024, hop_length=512)
      valid.append((S, len(s)))
  else:
    for f2 in sorted(glob.glob(path)):
      s, sr=librosa.load(f2, sr=None)
      S=librosa.stft(s, n_fft=1024, hop_length=512)
      valid.append((S, s))
  with open(picklepath, 'wb') as fi:
    pickle.dump(valid,fi)
  return(valid)

def testfilegen(path,picklepath):
  test = []
  for f3 in sorted(glob.glob(path)):
    s, sr=librosa.load(f3, sr=None)
    S=librosa.stft(s, n_fft=1024, hop_length=512)
    test.append((S, len(s),sr))
  with open(picklepath, 'wb') as fi:
    pickle.dump(test,fi)
  return(test)


####START *********************This portion for the loading of files takes 2 hours of time********************************

In [0]:
#Loading the training data
train_dirty = filegen('/content/drive/My Drive/homework3/timit-homework/tr/trx*','/content/drive/My Drive/homework3/train_dirty.pkl')
train_clean = filegen('/content/drive/My Drive/homework3/timit-homework/tr/trs*','/content/drive/My Drive/homework3/train_clean.pkl')
train_noise = filegen('/content/drive/My Drive/homework3/timit-homework/tr/trn*','/content/drive/My Drive/homework3/train_noise.pkl')

In [0]:
#Loading the validation data
valid_dirty = filegen('/content/drive/My Drive/homework3/timit-homework/v/vx*','/content/drive/My Drive/homework3/valid_dirty.pkl')
valid_clean = filegen('/content/drive/My Drive/homework3/timit-homework/v/vs*','/content/drive/My Drive/homework3/valid_clean.pkl')
valid_noise = filegen('/content/drive/My Drive/homework3/timit-homework/v/vn*','/content/drive/My Drive/homework3/valid_noise.pkl')

valid_dirty_main = validfilegen('/content/drive/My Drive/homework3/timit-homework/v/vx*','/content/drive/My Drive/homework3/valid_dirty_main.pkl')
valid_clean_main = validfilegen('/content/drive/My Drive/homework3/timit-homework/v/vs*','/content/drive/My Drive/homework3/valid_clean_main.pkl')

In [0]:
#Loading the test data
test_dirty = filegen('/content/drive/My Drive/homework3/timit-homework/te/tex*','/content/drive/My Drive/homework3/test_dirty.pkl')
test_dirty_main = testfilegen('/content/drive/My Drive/homework3/timit-homework/te/tex*','/content/drive/My Drive/homework3/test_dirty_main.pkl')


####END *********************This portion for the loading of files takes 2 hours of time********************************

Instead use the below portion!
I have added the pickled files to the submission

In [0]:

def load_pickle(path):
  with open(path, 'rb') as fi:
    fil = pickle.load(fi)
  return fil


In [0]:

#Loading the pickled training data
train_clean = load_pickle('/content/drive/My Drive/homework3/train_clean.pkl')
train_dirty = load_pickle('/content/drive/My Drive/homework3/train_dirty.pkl')
train_noise = load_pickle('/content/drive/My Drive/homework3/train_noise.pkl')

#Loading the pickled validation data
valid_clean = load_pickle('/content/drive/My Drive/homework3/valid_clean.pkl')
valid_dirty = load_pickle('/content/drive/My Drive/homework3/valid_dirty.pkl')
valid_noise = load_pickle('/content/drive/My Drive/homework3/valid_noise.pkl')
valid_dirty_main = load_pickle('/content/drive/My Drive/homework3/valid_dirty_main.pkl')
valid_clean_main = load_pickle('/content/drive/My Drive/homework3/valid_clean_main.pkl')

#Loading the pickled test data
test_dirty = load_pickle('/content/drive/My Drive/homework3/test_dirty.pkl')
test_dirty_main = load_pickle('/content/drive/My Drive/homework3/test_dirty_main.pkl')

In [0]:


def calc_IBM(signal,noise):
  ibm=np.greater(signal,noise)*1
  return(ibm)

def calc_snr(x,s):
  n = np.sum(np.square(x))
  d = np.sum(np.square(x-s))
  snr = 10* np.log10(n/d)
  return(snr)
  

The Ideal Binary Masks are being calculated here using the defined function, calc_IBM()

In [0]:
M = []
for i in range(1200):
  M.append(calc_IBM(train_clean[i],train_noise[i]))


In [0]:
import tensorflow as tf

In [0]:


x = tf.placeholder(tf.float32, [ None,None,np.shape(train_clean[0])[1]])
y = tf.placeholder(tf.float32, [None,None,np.shape(train_clean[0])[1]])

#### LSTM Model Description:
An LSTM cell with 513 units, having a dropout wrapper with dropout level(0.9) added for each RNN unit output.
At last an output dense layer with 513 units and a sigmoid activation is implemented.
Dynamic RNN to create a recurrent neural network, dynamically construct the graph when it is executed.
He initialization is used in the in the initial mentioned LSTM cell and in the output dense layer using variance_scaling_initializer module.
Mean Squared Error loss function and Adam optimizer are used with learning rate of 0.001.


In [0]:
#LSTM Model:

cell_lstm = tf.contrib.rnn.LSTMCell(513, initializer=tf.contrib.layers.variance_scaling_initializer())
dropout_lstm_layer = tf.nn.rnn_cell.DropoutWrapper(cell_lstm , output_keep_prob=0.9)
lstm_output_layer , state_lstm = tf.nn.dynamic_rnn(dropout_lstm_layer , x , dtype=tf.float32)
output_layer = tf.layers.dense(lstm_output_layer , 513 , activation=tf.nn.sigmoid , kernel_initializer=tf.contrib.layers.variance_scaling_initializer())


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [0]:
cost =tf.losses.mean_squared_error(labels=y, predictions=output_layer)
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)

In [0]:
n_epochs = 35
n_batch = 10
for epoch in range(n_epochs):
  epoch_loss = 0
  for i in range(0,1200,n_batch):
    epoch_x,epoch_y = np.array(train_dirty[i:i+n_batch]).reshape(n_batch,-1,513),np.array(M[i:i+n_batch]).reshape(n_batch,-1,513)
    _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
    epoch_loss += c
  print("Epoch",epoch,"loss",epoch_loss)

Epoch 0 loss 26.31024107336998
Epoch 1 loss 22.25534576177597
Epoch 2 loss 20.32280106842518
Epoch 3 loss 19.207242481410503
Epoch 4 loss 18.507744327187538
Epoch 5 loss 17.856200844049454
Epoch 6 loss 17.460377991199493
Epoch 7 loss 17.039649829268456
Epoch 8 loss 16.640930481255054
Epoch 9 loss 16.3269679620862
Epoch 10 loss 16.103137105703354
Epoch 11 loss 15.803884841501713
Epoch 12 loss 15.562882408499718
Epoch 13 loss 15.254823133349419
Epoch 14 loss 15.207882583141327
Epoch 15 loss 15.170404076576233
Epoch 16 loss 14.980057381093502
Epoch 17 loss 14.64017367362976
Epoch 18 loss 14.705688059329987
Epoch 19 loss 14.369383201003075
Epoch 20 loss 14.150674395263195
Epoch 21 loss 13.971485584974289
Epoch 22 loss 13.811717718839645
Epoch 23 loss 13.621875204145908
Epoch 24 loss 13.504915989935398
Epoch 25 loss 13.386620827019215
Epoch 26 loss 13.297039404511452
Epoch 27 loss 13.167198121547699
Epoch 28 loss 13.120986074209213
Epoch 29 loss 12.945222370326519
Epoch 30 loss 12.998136013

In [0]:

valid_clean_pred = []
for i in range(1200):
  M_val =  sess.run(output_layer, {x: valid_dirty[i].reshape(-1, valid_dirty[i].shape[0], 513)})
  M_val = np.multiply(M_val, valid_dirty[i])
  M_val = M_val.reshape(valid_dirty[i].shape[0], 513)
  hadamard_product = np.multiply(np.divide(valid_dirty_main[i][0], np.abs(valid_dirty_main[i][0])), M_val.T)
  pred_val = librosa.istft(hadamard_product, hop_length = 512, length = valid_dirty_main[i][1])
  valid_clean_pred.append(pred_val)
  

In [0]:

validation_snr = []
snr_val = 0
for i in range(len(valid_clean_main)):
  validation_snr.append(calc_snr(valid_clean_main[i][1], valid_clean_pred[i]))


In [0]:

print("The least SNR value in the validation set is: ",min(validation_snr))
print("The mean SNR value in the validation set is: ",np.mean(validation_snr))
print("The greatest SNR value in the validation set is: ",max(validation_snr))

The least SNR value in the validation set is:  2.4491603672504425
The mean SNR value in the validation set is:  10.569707577551405
The greatest SNR value in the validation set is:  24.29755449295044


The validation results, came out be having a <br>mean SNR value across the predictions by the LSTM network for the dirty validation signals to be 10.6

In [0]:

from google.colab import  files
test_clean_pred = []
for i in range(len(test_dirty)):
  M_test =  sess.run(output_layer, {x: test_dirty[i].reshape(-1, test_dirty[i].shape[0], 513)})
  M_test = np.multiply(M_test, test_dirty[i])
  M_test = M_test.reshape(test_dirty[i].shape[0], 513)
  hadamard_product = np.multiply(np.divide(test_dirty_main[i][0], np.abs(test_dirty_main[i][0])), M_test.T) 
  pred_test = librosa.istft(hadamard_product, hop_length = 512, length = test_dirty_main[i][1])
  librosa.output.write_wav('test_recon_'+str(i)+'.wav', pred_test, test_dirty_main[i][2])
  test_clean_pred.append(pred_test)


In [0]:
len(test_clean_pred)

400

The test clean reconstructed sounds are generated by the above code and the sound files would be downloaded by the below code.

In [0]:
 
 for i in range(len(test_clean_pred)):
   files.download('test_recon_' + str(i) + '.wav')  
   
print("Test reconstructed files generated and downloaded") 

Test reconstructed files generated and downloaded


The test files are downloaded and submitted as a zip folder.

The above code uses the colab imported files it sometimes fails to execute generating a Failed to Fetch exception, just need to re-run the cell.
A reported bug,


https://stackoverflow.com/questions/55713290/typeerror-failed-to-fetch-google-colab

In [0]:
import IPython.display as ipd
for i in range(len(test_clean_pred)):
  ipd.Audio('test_recon_'+str(i)+'.wav')